# Check whether the experimental settings match

In [3]:
import json
import os
import re
import ipynbname

path = os.path.dirname(ipynbname.path())
folder_path =  os.path.join(path,"prompt")
pattern = re.compile(r"(\d+)_character\.json$")

files = os.listdir(folder_path)

file_pairs = {}

# Find matching pairs of "character.json" and "game_setting_prompt.json"
for file_name in files:
    match = pattern.match(file_name)
    if match:
        prefix = match.group(1)
        character_file = file_name
        setting_file = f"{prefix}_game_setting_prompt.json"
        if setting_file in files:
            file_pairs[prefix] = (character_file, setting_file)

# Compare IDs between each pair of files
for prefix, (file1_name, file2_name) in file_pairs.items():
    file1_path = os.path.join(folder_path, file1_name)
    file2_path = os.path.join(folder_path, file2_name)

    with open(file1_path, "r", encoding="utf-8") as file1:
        data1 = json.load(file1)

    with open(file2_path, "r", encoding="utf-8") as file2:
        data2 = json.load(file2)

    ids1 = {item["id"] for item in data1}
    ids2 = {item["id"] for item in data2}

    # Check if the ID sets are different
    if ids1 != ids2:
        print(f"ID values are inconsistent between {prefix}_character.json and {prefix}_game_setting_prompt.json (prefix: {prefix})")

print("Check completed")


Check completed


# Check whether the prompt is complete

In [4]:
import json
import os
import re

path = os.path.dirname(ipynbname.path())
folder_path =  os.path.join(path,"prompt")
pattern = re.compile(r"(\d+)_game_setting_prompt.json$")

files = os.listdir(folder_path)

# Check whether the maximum value equals 60
for file_name in files:
    match = pattern.match(file_name)
    if match:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        max_trial = max(item["trial"] for item in data)
        if max_trial != 60:
            print(f"In {file_name}, the maximum item['trial'] value is {max_trial}, which does not meet the requirement.")

print("Check completed")


Check completed


# Check whether the output file format is correct

In [ ]:
import os
import pandas as pd

# List of models to check
models = ["gpt-3.5-turbo-0125", "o3-mini-2025-01-31", "deepseek-r1", "deepseek-v3"]
persona = "persona"
emotion = "emotion"
temperature = "1.0"

# Columns to check
required_cols = ["AA_valence", "AA_arousal", "choice", "AC_valence", "AC_arousal"]

def check_file(file_path):
    try:
        df = pd.read_csv(file_path, sep="\t", header=0)
    except Exception as e:
        return f"Failed to read: {e}"

    if len(df) != 60:
        return f"Incorrect number of rows: actual {len(df)}, expected 60"

    for col in required_cols:
        if col not in df.columns:
            return f"Missing column {col}"

    if df[required_cols].isnull().any().any():
        return "Missing values found in required columns"

    if not df["choice"].dropna().isin([0, 1]).all():
        return "'choice' column contains invalid values"

    for col in required_cols:
        if col != "choice":
            if not df[col].dropna().between(-100, 100).all():
                return f"Column {col} is out of range (-100, 100)"

    return "Format correct"


for model in models:
    path = os.path.dirname(ipynbname.path())
    folder_path =  os.path.join(path,f"result_{persona}_{emotion}_{temperature}_{model}")
    if not os.path.exists(folder_path):
        print(f"Folder does not exist: {folder_path}")
        continue

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            result = check_file(file_path)
            if result != "Format correct":
            #    os.remove(file_path)
                print(f"{file_name} in {model}: Format error -> {result}")

print("All files for all models have been checked")


All files for all models have been checked


# Check for missing agents

In [ ]:
import os

# List of models to check
models = ["gpt-3.5-turbo-0125", "o3-mini-2025-01-31", "deepseek-r1", "deepseek-v3"]
persona = "persona"
emotion = "emotion"
temperature = "1.0"

# Expected sample numbers
# Please note that this is only an example, which includes results from only two agents.
expected_numbers = range(0, 2)

for model in models:
    path = os.path.dirname(ipynbname.path())
    folder_path =  os.path.join(path,f"result_{persona}_{emotion}_{temperature}_{model}")

    if not os.path.isdir(folder_path):
        print(f"Folder does not exist: {folder_path}")
        continue

    missing_numbers = [
        n for n in expected_numbers
        if not os.path.isfile(os.path.join(folder_path, f"output_{n}.txt"))
    ]

    if missing_numbers:
        print(f"Missing samples in {model}: {missing_numbers}")
    else:
        print(f"All samples present in {model}")


All samples present in gpt-3.5-turbo-0125
All samples present in o3-mini-2025-01-31
All samples present in deepseek-r1
All samples present in deepseek-v3


# Merged the data

In [ ]:
import os
import json
import pandas as pd
from glob import glob

models = [
    "gpt-3.5-turbo-0125",
    "o3-mini-2025-01-31",
    "deepseek-r1",
    "deepseek-v3"
]

# Parameter definition
persona = "persona"
emotion = "emotion"
temperature = "1.0"

output_file = f"merged_all_models_{persona}_{emotion}_{temperature}.txt"
all_dfs = []

path = os.path.dirname(ipynbname.path())

for model in models:
    print(f"Processing model: {model}")
    
    prompt_folder = os.path.join(path,"prompt")
    result_folder = os.path.join(path,f"result_{persona}_{emotion}_{temperature}_{model}")

    prompt_files = glob(os.path.join(prompt_folder, "*_game_setting_prompt.json"))

    for prompt_file in prompt_files:
        prefix = os.path.basename(prompt_file).split('_')[0]
        result_file = os.path.join(result_folder, f"output_{prefix}.txt")
        if not os.path.exists(result_file):
            print(f"Result file for prefix {prefix} and model {model} does not exist. Skipping...")
            continue

        with open(prompt_file, 'r', encoding='utf-8') as f:
            prompt_data = json.load(f)
        prompt_df = pd.DataFrame(prompt_data)
        prompt_df['group'] = model    

        result_df = pd.read_csv(result_file, sep='\t', header=0)

        extra_cols = [c for c in prompt_df.columns if 'index' in c or c == 'Unnamed']
        if extra_cols:
            prompt_df = prompt_df.drop(columns=extra_cols)

        merged_df = pd.concat([prompt_df, result_df], axis=1)
        all_dfs.append(merged_df)

if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(output_file, sep='\t', index=False, encoding='utf-8')
    print(f"Results saved: {output_file}")


Processing model: gpt-3.5-turbo-0125
Processing model: o3-mini-2025-01-31
Processing model: deepseek-r1
Processing model: deepseek-v3
Results saved: merged_all_models_persona_emotion_1.0.txt
